In [36]:
from knn_model import *
pd.set_option('display.max_columns', None)
%matplotlib inline

In [67]:
    df = pd.read_csv('../data/new_olympics_merged.csv', sep = '|',lineterminator='\n')
    df_clean = prep_for_knn(df)
    test,train = train_test_split(df_clean,2016)
    #get weather data
    keys = ['Global_sum_FIPS:53031 FIPS:53009.csv','Global_sum_FIPS:53045 FIPS:53027.csv']
    df_weather = import_weather(keys)
    df_weather_dist = df_weather[['LATITUDE','LONGITUDE','name']].drop_duplicates().reset_index()
    #add feature columns

In [68]:
test

Date  last_year  year  condition|snow  condition|trail  \
0     2018-05-06       2017  2018            True             True   
1     2018-05-05       2017  2018           False             True   
2     2018-05-05       2017  2018            True             True   
3     2018-04-25       2017  2018            True             True   
4     2018-04-22       2017  2018           False             True   
5     2018-04-21       2017  2018            True            False   
6     2018-04-21       2017  2018           False            False   
7     2018-04-20       2017  2018            True             True   
8     2018-04-20       2017  2018            True             True   
9     2018-04-19       2017  2018            True             True   
10    2018-04-18       2017  2018            True             True   
11    2018-04-03       2017  2018            True             True   
12    2018-04-01       2017  2018            True            False   
13    2018-03-30       2017  2018            True             True   
14    2018-03-24       2017  2018            True             True   
15    2018-03-18       2017  2018            True             True   
16    2018-03-17       2017  2018            True             True   
17    2018-03-03       2017  2018           False            False   
18    2018-02-24       2017  2018            True             True   
19    2018-02-20       2017  2018            True             True   
20    2018-02-03       2017  2018            True             True   
21    2018-01-20       2017  2018            True            False   
22    2018-01-15       2017  2018            True            False   
23    2018-01-06       2017  2018            True            False   
24    2017-12-12       2016  2017            True             True   
25    2017-11-26       2016  2017           False             True   
26    2017-11-05       2016  2017            True            False   
27    2017-10-29       2016  2017           False            False   
28    2017-09-28       2016  2017           False            False   
29    2017-09-09       2016  2017           False            False   
...          ...        ...   ...             ...              ...   
12228 2016-04-17       2015  2016           False             True   
12229 2016-04-16       2015  2016           False             True   
12230 2016-04-12       2015  2016           False             True   
12231 2016-04-10       2015  2016           False             True   
12232 2016-04-10       2015  2016           False             True   
12233 2016-04-03       2015  2016           False             True   
12234 2016-04-01       2015  2016           False             True   
12235 2016-04-01       2015  2016           False             True   
12236 2016-03-31       2015  2016           False            False   
12237 2016-03-29       2015  2016           False             True   
12238 2016-03-27       2015  2016           False             True   
12239 2016-03-26       2015  2016           False             True   
12240 2016-03-25       2015  2016           False             True   
12241 2016-03-20       2015  2016           False             True   
12242 2016-03-20       2015  2016           False             True   
12243 2016-03-18       2015  2016           False             True   
12244 2016-03-16       2015  2016           False             True   
12245 2016-03-12       2015  2016           False             True   
12246 2016-03-06       2015  2016           False            False   
12247 2016-03-05       2015  2016           False            False   
12248 2016-02-28       2015  2016           False            False   
12249 2016-02-27       2015  2016           False            False   
12250 2016-02-03       2015  2016           False             True   
12251 2016-01-30       2015  2016           False             True   
12252 2016-01-26       2015  2016           False             True   
12253 2016-01-20      

In [69]:
conditions = ['condition|snow', 'condition|trail','condition|bugs','condition|road']
for condition in conditions:
    df = pd.read_csv('../data/new_olympics_merged.csv', sep = '|',lineterminator='\n')
    df_clean = prep_for_knn(df)
    test,train = train_test_split(df_clean,2016)
    #get weather data
    keys = ['Global_sum_FIPS:53031 FIPS:53009.csv','Global_sum_FIPS:53045 FIPS:53027.csv']
    df_weather = import_weather(keys)
    df_weather_dist = df_weather[['LATITUDE','LONGITUDE','name']].drop_duplicates().reset_index()
    
    add_cols(test,train, df_weather_dist,condition)
    #merge and save full df
    df_test = merge_weather_trails(df_weather,test)
    df_train = merge_weather_trails(df_weather,train)
    train_X,train_y,test_X,test_y = get_knn_inputs(df_test,df_train,condition)
    model,pred = make_forest(train_X,train_y,test_X,test_y)
    
    y_true = test_y
    y_pred = pred[:,1]

    fpr, tpr, threshold = roc_curve(y_true, y_pred)
    area_under_curve = auc(fpr, tpr)
    print (condition, area_under_curve)

#     # method I: plt
#     fig, ax = plt.subplots()
#     plt.title(f'Receiver Operating Characteristic - {condition}')
#     ax.plot(fpr, tpr, 'k', label = 'C AUC = %0.3f' % area_under_curve)

#     ax.legend(loc = 'lower right')
#     ax.plot([0, 1], [0, 1],'r--')
#     plt.xlim([0, 1])
#     plt.ylim([0, 1])
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.show()
    test.drop([f'neighbors_average {condition}'],axis=1)
    train.drop([f'neighbors_average {condition}'],axis=1)

/Users/ashley/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/ashley/projects/Trail_Report/src/knn_model.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[f'neighbors_average {condition}'] = averages
/Users/ashley/projects/Trail_Report/src/Cleaning/Merge_Weather.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

condition|snow 1.0


/Users/ashley/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/ashley/projects/Trail_Report/src/knn_model.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[f'neighbors_average {condition}'] = averages
/Users/ashley/projects/Trail_Report/src/Cleaning/Merge_Weather.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

condition|trail 1.0


/Users/ashley/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/ashley/projects/Trail_Report/src/knn_model.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[f'neighbors_average {condition}'] = averages
/Users/ashley/projects/Trail_Report/src/Cleaning/Merge_Weather.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

condition|bugs 0.999918082888


/Users/ashley/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/Users/ashley/projects/Trail_Report/src/knn_model.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[f'neighbors_average {condition}'] = averages
/Users/ashley/projects/Trail_Report/src/Cleaning/Merge_Weather.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

condition|road 1.0


In [21]:
model,pred = make_forest(train_X,train_y,test_X,test_y)
model.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [22]:
y_true = test_y
y_pred = pred[:,1]

fpr, tpr, threshold = roc_curve(y_true, y_pred)
area_under_curve = auc(fpr, tpr)

# method I: plt
fig, ax = plt.subplots()
plt.title('Receiver Operating Characteristic')
ax.plot(fpr, tpr, 'k', label = 'C AUC = %0.3f' % area_under_curve)

ax.legend(loc = 'lower right')
ax.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [24]:
feature_values = {}
for i in range(len(model.feature_importances_)):
   feature_values[test_X.columns[i]] = model.feature_importances_[i]

top_features = sorted(feature_values.items(), key=lambda x: x[1], reverse=True)
top_features

[('neighbors_average', 0.30159563381944338),
 ('month', 0.23499719875040045),
 ('highest_point', 0.028135668549925365),
 ('elevation_gain', 0.023615252472676578),
 ('long', 0.018999243320777657),
 ('total_distance', 0.017861115134421599),
 ('lat', 0.017653445156102624),
 ('stars', 0.01712456075486795),
 ('numReports', 0.017115053311486404),
 ('DP01', 0.015899374022125226),
 ('number_votes', 0.015716333116401856),
 ('DP10', 0.013637146292816866),
 ('PRCP', 0.012218047784845363),
 ('EMXP', 0.010290690274848359),
 ('TMAX', 0.00895838773143285),
 ('HDSD', 0.0087387047871515973),
 ('HTDD', 0.0087193237195635192),
 ('DX70', 0.0080000027392653581),
 ('EMXT', 0.0067508993280005038),
 ('TAVG', 0.0061342448518979771),
 ('Ridges/passes', 0.0053079302337229661),
 ('FZF0', 0.0051620654885430692),
 ('Mountain views', 0.0051195780999764165),
 ('Wildflowers/Meadows', 0.0049216549208733361),
 ('Lakes', 0.0049154268071376879),
 ('Rivers', 0.0048925814389208537),
 ('Established campsites', 0.004694102299

In [30]:
conditions = ['condition|snow', 'condition|trail','condition|bugs',
    'condition|road']

In [31]:
conditon = 'condition|snow'

In [33]:
conditions.remove(conditon)

In [51]:
type(conditions)

list

In [52]:
more = ['test','this','one']
type(more+conditions)

list